# Analyses des modèles


Voir

- <https://python.quantecon.org/mle.html>

In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf


from scipy import stats
from sklearn.decomposition import TruncatedSVD

from IPython.display import display

from catk import CA
from catk.data import get_colours, get_smokers, get_wisconsin

np.set_printoptions(precision=3)


'/home/romulus/Documents/unc-informatique.pharmaco-chemistry-biblio/catk/catk/data/data.py' loaded
'/home/romulus/Documents/unc-informatique.pharmaco-chemistry-biblio/catk/catk/data/__init__.py' loaded
'/home/romulus/Documents/unc-informatique.pharmaco-chemistry-biblio/catk/catk/ca.py' loaded
'/home/romulus/Documents/unc-informatique.pharmaco-chemistry-biblio/catk/catk/__init__.py' loaded


In [2]:
# df = get_colours()
df = get_smokers()
ca = CA().fit(df)
M = df.to_numpy()
df


Smoking category,None,Light,Medium,Heavy
Staff group,,,,
Senior Managers,4,2,3,2
Junior Managers,4,3,7,4
Senior Employees,25,10,12,4
Junior Employees,18,24,33,13
Secretaries,10,6,7,2


In [3]:
print("n =", n := ca.n)
print("I =", I := ca.I)
print("J =", J := ca.J)

r, c = stats.contingency.margins(M)
# c := M.sum(axis=0).reshape(1, -1) / n
# r := M.sum(axis=1).reshape(-1, 1) / n
print("c =", c)
print("r =", r)


n = 193
I = 5
J = 4
c = [[61 45 62 25]]
r = [[11]
 [18]
 [51]
 [88]
 [25]]


In [4]:
stats.contingency.chi2_contingency(M)

(
    16.441643072223254,
    0.17183477869584723,
    12,
    array([[ 3.477,  2.565,  3.534,  1.425],
       [ 5.689,  4.197,  5.782,  2.332],
       [16.119, 11.891, 16.383,  6.606],
       [27.813, 20.518, 28.269, 11.399],
       [ 7.902,  5.829,  8.031,  3.238]])
)

Pour um modèle Poisson, le paramètre est _l'effectif_ de la case $c_{ij}$.

Ici, on ne donne :

- _null model_, qu'un seul paramètre global $\mu_{ij} = \eta$, chaque a le même
- _saturate model_, autant de paramètres que de cases, avec $\mu_{ij} = \eta + \beta_i + \beta_j + \beta_{ij}$

En modèle Poisson, le _grand total_ n'est pas fixe, car c'est la somme des Poissons, qui a pour moyenne ... ?

In [5]:
null_mu = np.ones_like(M) * n / M.size
display(null_mu)
null_model = stats.poisson(null_mu)
# on tire un jeu selon ce modèle
display(S := null_model.rvs())
print(f"n = {S.sum()}")

array([[9.65, 9.65, 9.65, 9.65],
       [9.65, 9.65, 9.65, 9.65],
       [9.65, 9.65, 9.65, 9.65],
       [9.65, 9.65, 9.65, 9.65],
       [9.65, 9.65, 9.65, 9.65]])

array([[12, 10,  9,  9],
       [ 9, 11, 12,  7],
       [ 8,  9, 11, 11],
       [ 7,  9, 11,  8],
       [10, 14,  9, 13]])

n = 199


In [6]:
# print(f"dof = {r.size} + {c.size} + 1 - 2 = {r.size + c.size + 1 - 2}")
saturated_mu = r @ c / n
display(saturated_mu)
# saturated Poisson
saturated_model = stats.poisson(saturated_mu)
# on tire un jeu selon ce modèle
display(S := saturated_model.rvs())
print(f"n = {S.sum()}")


array([[ 3.477,  2.565,  3.534,  1.425],
       [ 5.689,  4.197,  5.782,  2.332],
       [16.119, 11.891, 16.383,  6.606],
       [27.813, 20.518, 28.269, 11.399],
       [ 7.902,  5.829,  8.031,  3.238]])

array([[ 1,  2,  3,  2],
       [ 5,  3,  3,  5],
       [12, 13, 18,  7],
       [39, 19, 36, 13],
       [10,  6, 13,  3]])

n = 213


In [7]:
# on compare la proba de tirer cette matrice par rapport au modèle E

P_S = saturated_model.pmf(S).prod()
print(f"P(X = S) = {P_S}")
E = (n * r @ c).astype(np.uint64)
P_E = saturated_model.pmf(E).prod()
print(f"P(X = E) = {P_E}")
print(f"P(X = S)/P(X = E) = {100*P_S/P_E:.2f}%")


P(X = S) = 4.035878093694699e-21
P(X = E) = 0.0
P(X = S)/P(X = E) = inf%


/tmp/ipykernel_1660486/454954508.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  print(f"P(X = S)/P(X = E) = {100*P_S/P_E:.2f}%")


In [8]:
# on compare la proba de tirer la matrice d'origine perturbée par Poisson(0.2)

perturb = stats.poisson(mu=np.array([0.2]*M.size).reshape(M.shape))
Ep = E + perturb.rvs()
display(Ep - E)
P_Ep = saturated_model.pmf(Ep).prod()
print(f"P(X = Ep) = {P_Ep}")
print(f"P(X = Ep)/P(X = E) = {100*P_Ep/P_E:.2f}%")



array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

P(X = Ep) = 0.0
P(X = Ep)/P(X = E) = nan%


/tmp/ipykernel_1660486/2284770072.py:8: RuntimeWarning: invalid value encountered in double_scalars
  print(f"P(X = Ep)/P(X = E) = {100*P_Ep/P_E:.2f}%")


In [9]:
K = 1  # min(I,J)
mask = np.eye(I, J) * ([1] * K + [0] * (min(I, J) - K))
A = ((ca.U @ (ca.Da * mask) @ ca.Vt) * np.sqrt(c) * np.sqrt(r) + r @ c)
n*(A - M/n)


array([[ 129562.481,   95521.158,  131590.794,   53053.567],
       [ 211500.99 ,  156403.298,  215588.505,   86971.206],
       [ 602101.194,  442607.28 ,  609389.909,  245549.616],
       [1034207.212,  764591.552, 1053887.589,  425137.647],
       [ 294756.123,  217036.711,  298919.203,  120487.963]])

Pour $K=1$, ici $J < I$ on a :

- 1  paramètre pour $n$
- $(I-1)$ paramètres pour $r$
- $(J-1)$ paramètres pour $c$
- $K*(I-1)$ paramètres pour le premier axe

Comme si :

- on gardait la première _ligne_ de $V^T$
- on gardait la première _colonne_ de $U$

In [10]:
display(mask @ ca.Vt)
display(ca.U @ mask)
display(ca.U @ (ca.Da * mask) @ ca.Vt)
# display(ca.U @ (mask) @ ca.Vt)
# on reconstruit à partir de la première ligne de Vt et colonne de U
R = ca.U[:, 0].reshape(I,1) @ ca.Vt[0,:].reshape(1,J)
display(R*ca.eiv[0])

array([[-0.809,  0.176,  0.407,  0.387],
       [ 0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ]])

array([[-0.057,  0.   ,  0.   ,  0.   ],
       [ 0.289,  0.   ,  0.   ,  0.   ],
       [-0.716,  0.   ,  0.   ,  0.   ],
       [ 0.575,  0.   ,  0.   ,  0.   ],
       [-0.265,  0.   ,  0.   ,  0.   ]])

array([[ 0.013, -0.003, -0.006, -0.006],
       [-0.064,  0.014,  0.032,  0.031],
       [ 0.158, -0.034, -0.08 , -0.076],
       [-0.127,  0.028,  0.064,  0.061],
       [ 0.059, -0.013, -0.029, -0.028]])

array([[ 0.013, -0.003, -0.006, -0.006],
       [-0.064,  0.014,  0.032,  0.031],
       [ 0.158, -0.034, -0.08 , -0.076],
       [-0.127,  0.028,  0.064,  0.061],
       [ 0.059, -0.013, -0.029, -0.028]])

Si on sait passer de $V^T$ à $U$ (ou l'inverse, selon la plsu grande dimension) on a gagné

In [11]:
# la mm en coord std
(np.sqrt(r) @ np.sqrt(c))**2 - r@c


array([[ 0.000e+00,  0.000e+00,  1.137e-13, -5.684e-14],
       [-4.547e-13,  0.000e+00, -2.274e-13, -1.137e-13],
       [ 0.000e+00,  4.547e-13,  0.000e+00,  2.274e-13],
       [-9.095e-13,  4.547e-13,  0.000e+00,  4.547e-13],
       [-2.274e-13,  4.547e-13,  0.000e+00,  0.000e+00]])

## Modèle loglinear

On va refaire le tableau de "Log Linear Models for Contingency Tables"

In [125]:
df = get_wisconsin(False)
df_items = df.melt(ignore_index=False).reset_index()
columns = list("SEP")
df_items.columns = columns + ["Count"]
df_items = df_items.sort_values(by=columns)
display(df_items)


,S,E,P,Count
1,Lower,High,No,233
9,Lower,High,Yes,133
0,Lower,Low,No,749
8,Lower,Low,Yes,35
3,Lower Middle,High,No,330
11,Lower Middle,High,Yes,303
2,Lower Middle,Low,No,627
10,Lower Middle,Low,Yes,38
7,Upper,High,No,266
15,Upper,High,Yes,800


In [99]:
model = smf.glm("Count ~ S + E + P - 1", data=df_items, family=sm.families.Poisson(sm.families.links.Log())).fit()
print(model.deviance, model.df_resid)
model.summary()


2713.953831980395 10


Generalized Linear Model Regression Results 
 
 Dep. Variable: Count No. Observations: 16 
 
 
 Model: GLM Df Residuals: 10 
 
 
 Model Family: Poisson Df Model: 5 
 
 
 Link Function: Log Scale: 1.0000 
 
 
 Method: IRLS Log-Likelihood: -1413.9 
 
 
 Date: Fri, 13 May 2022 Deviance: 2714.0 
 
 
 Time: 10:55:45 Pearson chi2: 3.00e+03 
 
 
 No. Iterations: 5 Pseudo R-squ. (CS): 1.000 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err z P>|z| [0.025 0.975] 
 
 
 S[Lower] 6.0471 0.034 179.872 0.000 5.981 6.113 
 
 
 S[Lower Middle] 6.1681 0.032 192.092 0.000 6.105 6.231 
 
 
 S[Upper] 6.1264 0.033 187.830 0.000 6.063 6.190 
 
 
 S[Upper Middle] 6.1681 0.032 192.092 0.000 6.105 6.231 
 
 
 E[T.Low] -0.3320 0.029 -11.568 0.000 -0.388 -0.276 
 
 
 P[T.Yes] -0.5388 0.029 -18.362 0.000 -0.596 -0.481

In [126]:
nb_products = len(columns) - 1  # sans 1 : saturé


def gen_models():
    all_pairs = set(combinations(columns, 2))
    all_models = []
    for i in range(len(all_pairs) + 1):
        # print(f"len {i}: {list(combinations(all_pairs, i))}")
        all_models.extend(list(l) for l in combinations(all_pairs, i))

    # complete
    for choice in all_models:
        used = set(a for c in choice for a in c)
        free = set(columns) - used
        choice.extend((f,) for f in free)
    return all_models # [sorted(sorted(p) for p in m) for m in all_models]

gen_models()

[
    [('P',), ('S',), ('E',)],
    [('S', 'E'), ('P',)],
    [('E', 'P'), ('S',)],
    [('S', 'P'), ('E',)],
    [('S', 'E'), ('E', 'P')],
    [('S', 'E'), ('S', 'P')],
    [('E', 'P'), ('S', 'P')],
    [('S', 'E'), ('E', 'P'), ('S', 'P')]
]

In [127]:
res = {}
for model in gen_models():
    formula = " + ".join(" * ".join(term) for term in model)  # * or : is the same ?
    full_formula = f"Count ~ {formula}"
    print(full_formula)
    model = smf.glm(full_formula, data=df_items, family=sm.families.Poisson(sm.families.links.Log())).fit()
    # display(model.summary2())
    res[formula] = (model.deviance, model.df_resid, model.df_model)

res


Count ~ P + S + E
Count ~ S * E + P
Count ~ E * P + S
Count ~ S * P + E
Count ~ S * E + E * P
Count ~ S * E + S * P
Count ~ E * P + S * P
Count ~ S * E + E * P + S * P


{
    'P + S + E': (2713.9538319803946, 10, 5),
    'S * E + P': (1877.3816276612977, 7, 8),
    'E * P + S': (1092.039989856524, 9, 6),
    'S * P + E': (1920.398999604215, 7, 8),
    'S * E + E * P': (255.46778553742604, 6, 9),
    'S * E + S * P': (1083.826795285117, 4, 11),
    'E * P + S * P': (298.4851574803441, 6, 9),
    'S * E + E * P + S * P': (1.5754679838712584, 3, 12)
}

In [128]:
# for m, (dev, dof_resid, dof_model) in sorted(res.items(), key=lambda x:-x[1][0]):
#     print(m, dev, dof_resid, dof_model)

df_res = pd.DataFrame(((k,) + v for k, v in res.items()), columns=["Model", "Deviance", "DoF residuals", "DoF model"]).set_index("Model")
pd.set_option("display.precision", 1)
display(df_res)


,Deviance,DoF residuals,DoF model
Model,,,
P + S + E,2714.0,10,5
S * E + P,1877.4,7,8
E * P + S,1092.0,9,6
S * P + E,1920.4,7,8
S * E + E * P,255.5,6,9
S * E + S * P,1083.8,4,11
E * P + S * P,298.5,6,9
S * E + E * P + S * P,1.6,3,12


In [129]:
# arr = m.sort_values(by= columns)["Count"].to_numpy().reshape(4,2,2)
# stats.chi2_contingency(arr, lambda_="log-likelihood")
ref_dev = df_res.loc["P + S + E", "Deviance"]
df_res["Delta dev."] = (ref_dev - df_res["Deviance"])
df_res["Delta dev./dof model"] = df_res["Delta dev."]/df_res["DoF model"]
df_res["Dev./dof res"] = df_res["Deviance"]/df_res["DoF residuals"]
display(df_res)


,Deviance,DoF residuals,DoF model,Delta dev.,Delta dev./dof model,Dev./dof res
Model,,,,,,
P + S + E,2714.0,10,5,0.0,0.0,271.4
S * E + P,1877.4,7,8,836.6,104.6,268.2
E * P + S,1092.0,9,6,1621.9,270.3,121.3
S * P + E,1920.4,7,8,793.6,99.2,274.3
S * E + E * P,255.5,6,9,2458.5,273.2,42.6
S * E + S * P,1083.8,4,11,1630.1,148.2,271.0
E * P + S * P,298.5,6,9,2415.5,268.4,49.7
S * E + E * P + S * P,1.6,3,12,2712.4,226.0,0.5


In [130]:
uniform_model = model # the last one
pred = uniform_model.predict().reshape(4,2,2)
df_items["Pred"] = pred.flatten()
df_items.pivot(index = ["S", 'E'], columns=["P"], values="Pred")

P 
 No 
 Yes 
 
 
 S 
 E 
 
 
 
 
 
 
 Lower 
 High 
 228.9 
 137.1 
 
 
 Low 
 753.1 
 30.9 
 
 
 Lower Middle 
 High 
 331.0 
 302.0 
 
 
 Low 
 626.0 
 39.0 
 
 
 Upper 
 High 
 270.0 
 796.0 
 
 
 Low 
 149.0 
 30.0 
 
 
 Upper Middle 
 High 
 373.1 
 467.9 
 
 
 Low 
 420.9 
 36.1

In [132]:
df["Pr"] = df["Yes"]/df.sum(axis=1)
df

College Plans 
 No 
 Yes 
 Pr 
 
 
 
 
 Lower 
 Low 
 749 
 35 
 4.4e-02 
 
 
 High 
 233 
 133 
 3.3e-01 
 
 
 Lower Middle 
 Low 
 627 
 38 
 5.7e-02 
 
 
 High 
 330 
 303 
 4.5e-01 
 
 
 Upper Middle 
 Low 
 420 
 37 
 8.0e-02 
 
 
 High 
 374 
 467 
 5.2e-01 
 
 
 Upper 
 Low 
 153 
 26 
 1.3e-01 
 
 
 High 
 266 
 800 
 7.0e-01

In [142]:
df_log = df.drop(columns=["Yes", "No"])
df_log = df_log.melt(ignore_index=False).reset_index().drop(columns=["College Plans"])
df_log.columns = ["S", "E", "Pr"]
df_log

,S,E,Pr
0,Lower,Low,4.4e-02
1,Lower,High,3.3e-01
2,Lower Middle,Low,5.7e-02
3,Lower Middle,High,4.5e-01
4,Upper Middle,Low,8.0e-02
5,Upper Middle,High,5.2e-01
6,Upper,Low,1.3e-01
7,Upper,High,7.0e-01


Logit

In [149]:
# model = smf.logit("Pr ~ S + E", data=df_log).fit()
model = smf.glm("Pr ~ 1", data=df_log, family=sm.families.Binomial()).fit()
display(model.summary())
model.predict()


Generalized Linear Model Regression Results 
 
 Dep. Variable: Pr No. Observations: 8 
 
 
 Model: GLM Df Residuals: 7 
 
 
 Model Family: Binomial Df Model: 0 
 
 
 Link Function: Logit Scale: 1.0000 
 
 
 Method: IRLS Log-Likelihood: -3.6304 
 
 
 Date: Fri, 13 May 2022 Deviance: 2.2302 
 
 
 Time: 11:46:18 Pearson chi2: 2.10 
 
 
 No. Iterations: 4 Pseudo R-squ. (CS): 2.220e-16 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err z P>|z| [0.025 0.975] 
 
 
 Intercept -0.8999 0.780 -1.154 0.249 -2.428 0.629

array([0.289, 0.289, 0.289, 0.289, 0.289, 0.289, 0.289, 0.289])

In [152]:
model = smf.logit("Pr ~ 1", data=df_log).fit()
display(model.summary())
model.predict()

Optimization terminated successfully.
         Current function value: 0.542296
         Iterations 5


Logit Regression Results 
 
 Dep. Variable: Pr No. Observations: 8 
 
 
 Model: Logit Df Residuals: 7 
 
 
 Method: MLE Df Model: 0 
 
 
 Date: Fri, 13 May 2022 Pseudo R-squ.: -0.2820 
 
 
 Time: 11:48:08 Log-Likelihood: -4.3384 
 
 
 converged: True LL-Null: -3.3841 
 
 
 Covariance Type: nonrobust LLR p-value: nan 
 
 
 
 
 coef std err z P>|z| [0.025 0.975] 
 
 
 Intercept -0.8999 0.780 -1.154 0.249 -2.428 0.629

array([0.289, 0.289, 0.289, 0.289, 0.289, 0.289, 0.289, 0.289])